In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#작업 경로 지정
import os
os.chdir('/content/drive/MyDrive/woke-odds')
print(os.getcwd())

/content/drive/MyDrive/woke-odds


In [ ]:
import pandas as pd

df = pd.read_parquet('clarification_response.parquet')

In [ ]:
print(f"전체 데이터 개수: {len(df)}")
print(df.head())

전체 데이터 개수: 1601
                                               query  \
0     [AO|WHOM] Give me a list of good coffee shops?   
1    [AO|WHOM] Give me some Mother's Day gift ideas.   
2  [AO|WHAT] Help me come up with 3 ideas for a n...   
3  [AO|WHOM] Write the first paragraph of a blog ...   
4  [AO|WHOM] Give me some tips on how to train fo...   

                                          question_1  \
0  >\n\n\n\nQuestion: What is the city or country...   
1              >\n\n\n\nWhen do you come in with me?   
2   WHAT is one of the businesses you want to start?   
3                WHAT was the name of his first dog?   
4  I'm not sure if you're asking for the training...   

                                          question_2 category subclass  \
0                                WHOM is a location?       AO     WHOM   
1  >\n\nWhat type of gift do you mean, such as pe...       AO     WHOM   
2  <think>\nOkay, the user's query is about needi...       AO     WHAT   
3             

In [ ]:
! git config --global credential.helper store

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `NLP_HW` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured 

In [ ]:
!hf auth whoami

user:  jyering


In [ ]:
!pip install -q transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "jyering/clarifying_sft_phi"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.62k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.91M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/568 [00:00<?, ?B/s]

In [ ]:
# 각 답변의 문자열 길이 계산
df['question_1_length'] = df['question_1'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
df['question_2_length'] = df['question_2'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)

In [ ]:
threshold = 100
long_q1_count = (df['question_1_length'] > threshold).sum()
long_q2_count = (df['question_2_length'] > threshold).sum()

print(f"{threshold}자 이상 답변 개수")
print(f"Question 1: {long_q1_count}개")
print(f"Question 2: {long_q2_count}개")

100자 이상 답변 개수
Question 1: 262개
Question 2: 246개


In [ ]:
threshold = 50
long_q1_count = (df['question_1_length'] > threshold).sum()
long_q2_count = (df['question_2_length'] > threshold).sum()

print(f"{threshold}자 이상 답변 개수")
print(f"Question 1: {long_q1_count}개")
print(f"Question 2: {long_q2_count}개")

50자 이상 답변 개수
Question 1: 550개
Question 2: 537개


In [ ]:
# 각 답변의 토큰 수 계산
df['question_1_tokens'] = df['question_1'].apply(
    lambda x: len(tokenizer.encode(str(x))) if pd.notna(x) else 0
)
df['question_2_tokens'] = df['question_2'].apply(
    lambda x: len(tokenizer.encode(str(x))) if pd.notna(x) else 0
)

In [ ]:
threshold = 100
long_q1 = df[df['question_1_tokens'] > threshold]
long_q2 = df[df['question_2_tokens'] > threshold]

print(f"토큰 수 {threshold} 이상인 데이터")
print(f"Question 1: {len(long_q1)}개 ({len(long_q1)/len(df)*100:.2f}%)")
print(f"Question 2: {len(long_q2)}개 ({len(long_q2)/len(df)*100:.2f}%)")

토큰 수 100 이상인 데이터
Question 1: 161개 (10.06%)
Question 2: 158개 (9.87%)


In [ ]:
threshold = 50
long_q1 = df[df['question_1_tokens'] > threshold]
long_q2 = df[df['question_2_tokens'] > threshold]

print(f"토큰 수 {threshold} 이상인 데이터")
print(f"Question 1: {len(long_q1)}개 ({len(long_q1)/len(df)*100:.2f}%)")
print(f"Question 2: {len(long_q2)}개 ({len(long_q2)/len(df)*100:.2f}%)")

토큰 수 50 이상인 데이터
Question 1: 179개 (11.18%)
Question 2: 177개 (11.06%)


In [ ]:
# 3. 개행 문자(\n) 포함 개수
newline_q1_count = df['question_1'].apply(lambda x: '\n' in str(x) if pd.notna(x) else False).sum()
newline_q2_count = df['question_2'].apply(lambda x: '\n' in str(x) if pd.notna(x) else False).sum()

print(f"개행 문자(\\n) 포함 개수")
print(f"Question 1: {newline_q1_count}개")
print(f"Question 2: {newline_q2_count}개")

개행 문자(\n) 포함 개수
Question 1: 528개
Question 2: 500개


토큰 수 50 이상 답변 후처리

In [ ]:
long_q1.head()

,query,question_1,question_2,category,subclass,original_question,question_1_length,question_2_length,question_1_tokens,question_2_tokens
10,[AO|WHAT] How do I optimize my Kafka consumpti...,"<think>\nOkay, so the user is asking how to op...","<think>\nOkay, user mentioned optimizing Kafka...",AO,WHAT,How do I optimize my Kafka consumption in Spar...,5230,5184,1048,1047
23,[AO|WHAT] Tell me whether these states are in ...,<think> In this case we need to output a clari...,WHAT are European country's capitals?,AO,WHAT,Tell me whether these states are in Eastern Ti...,4980,37,1047,6
33,[AO|WHOM] How difficult is it to learn how to ...,"<think>\nOkay, so the user here is asking abou...",In case you have multiple difficulty options? ...,AO,WHOM,How difficult is it to learn how to water ski?,4810,154,1048,31
35,[AO|WHOM] Provide a list of some Japanese art ...,"<think>\n\nOkay, I need to generate a clarifyi...","<think>\nOkay, let's see. The user mentioned s...",AO,WHOM,Provide a list of some Japanese art forms I've...,5110,4762,1048,1048
37,[AO|WHOM] Provide a 10-step flow for a heteros...,"<think>\n\nOkay, so I need to generate a clari...",WHAT IS THE LAY-OUT?,AO,WHOM,Provide a 10-step flow for a heterosexual coup...,4847,20,1047,8


In [ ]:
long_q2.head()

,query,question_1,question_2,category,subclass,original_question,question_1_length,question_2_length,question_1_tokens,question_2_tokens
2,[AO|WHAT] Help me come up with 3 ideas for a n...,WHAT is one of the businesses you want to start?,"<think>\nOkay, the user's query is about needi...",AO,WHAT,Help me come up with 3 ideas for a new busines...,48,4689,11,934
4,[AO|WHOM] Give me some tips on how to train fo...,I'm not sure if you're asking for the training...,"<think>\nOkay, I need to generate a clarifying...",AO,WHOM,Give me some tips on how to train for a marathon.,64,4928,14,1046
10,[AO|WHAT] How do I optimize my Kafka consumpti...,"<think>\nOkay, so the user is asking how to op...","<think>\nOkay, user mentioned optimizing Kafka...",AO,WHAT,How do I optimize my Kafka consumption in Spar...,5230,5184,1048,1047
25,[AO|WHOM] What to do when you are bored?,---\n\nWhen to do what you need to do?,"<think>\nOkay, I need to generate a clarifying...",AO,WHOM,What to do when you are bored?,36,2581,10,635
28,[AO|WHOM] Give me a list of 3 of the best movi...,>Please specify which type of quotes you would...,"<think>\nOkay, I need to generate a clarifying...",AO,WHOM,Give me a list of 3 of the best movies quotes ...,63,4390,12,875


In [ ]:
print(f"토큰 수 {threshold} 초과:")
print(f"Question 1: {len(long_q1)}개")
print(f"Question 2: {len(long_q2)}개\n")

토큰 수 50 초과:
Question 1: 179개
Question 2: 177개



In [ ]:
# /n 제거 & 연속 공백 제거
import re

def remove_newlines_and_spaces(text):
    if pd.isna(text):
        return text

    text = str(text)
    # 개행 제거
    text = text.replace('\n', ' ').replace('\r', ' ')
    # 연속 공백 제거
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

#적용
long_q1['question_1_no_newline'] = long_q1['question_1'].apply(remove_newlines_and_spaces)
long_q2['question_2_no_newline'] = long_q2['question_2'].apply(remove_newlines_and_spaces)

/tmp/ipython-input-1631015692.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_q1['question_1_no_newline'] = long_q1['question_1'].apply(remove_newlines_and_spaces)
/tmp/ipython-input-1631015692.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_q2['question_2_no_newline'] = long_q2['question_2'].apply(remove_newlines_and_spaces)


In [ ]:
long_q1.head(3)

,query,question_1,question_2,category,subclass,original_question,question_1_length,question_2_length,question_1_tokens,question_2_tokens,question_1_no_newline
10,[AO|WHAT] How do I optimize my Kafka consumpti...,"<think>\nOkay, so the user is asking how to op...","<think>\nOkay, user mentioned optimizing Kafka...",AO,WHAT,How do I optimize my Kafka consumption in Spar...,5230,5184,1048,1047,"<think> Okay, so the user is asking how to opt..."
23,[AO|WHAT] Tell me whether these states are in ...,<think> In this case we need to output a clari...,WHAT are European country's capitals?,AO,WHAT,Tell me whether these states are in Eastern Ti...,4980,37,1047,6,<think> In this case we need to output a clari...
33,[AO|WHOM] How difficult is it to learn how to ...,"<think>\nOkay, so the user here is asking abou...",In case you have multiple difficulty options? ...,AO,WHOM,How difficult is it to learn how to water ski?,4810,154,1048,31,"<think> Okay, so the user here is asking about..."


In [ ]:
long_q2.head(3)

,query,question_1,question_2,category,subclass,original_question,question_1_length,question_2_length,question_1_tokens,question_2_tokens,question_2_no_newline
2,[AO|WHAT] Help me come up with 3 ideas for a n...,WHAT is one of the businesses you want to start?,"<think>\nOkay, the user's query is about needi...",AO,WHAT,Help me come up with 3 ideas for a new busines...,48,4689,11,934,"<think> Okay, the user's query is about needin..."
4,[AO|WHOM] Give me some tips on how to train fo...,I'm not sure if you're asking for the training...,"<think>\nOkay, I need to generate a clarifying...",AO,WHOM,Give me some tips on how to train for a marathon.,64,4928,14,1046,"<think> Okay, I need to generate a clarifying ..."
10,[AO|WHAT] How do I optimize my Kafka consumpti...,"<think>\nOkay, so the user is asking how to op...","<think>\nOkay, user mentioned optimizing Kafka...",AO,WHAT,How do I optimize my Kafka consumption in Spar...,5230,5184,1048,1047,"<think> Okay, user mentioned optimizing Kafka ..."


In [ ]:
#<think> 태그에 따라 답변 추출
def extract_final_answer(text):
    """
    <think>로 시작하는 경우:
    - </>로 끝나면: </> 이후 내용을 답변으로
    - </>로 안 끝나면: 마지막 ?이 포함된 문장을 답변으로
    <think>로 시작하지 않으면: 전체를 답변으로
    """
    if pd.isna(text):
        return text

    text = str(text).strip()

    # <think>로 시작하는지 확인
    if text.startswith('<think>'):
      # 케이스 1: </>로 끝나는 경우
      if '</>' in text:
        # </> 이후의 내용 추출
        parts = text.split('</>')
        if len(parts) > 1:
          answer = parts[-1].strip()
          return answer if answer else text
        return text

      # 케이스 2: </>로 안 끝나는 경우
      else:
        # 문장 분리 (? 기준)
        sentences = re.split(r'(?<=[.!?])\s+', text)

        # 마지막부터 역순으로 ?가 포함된 문장 찾기
        for sentence in reversed(sentences):
          if '?' in sentence:
            return sentence.strip()

        # ?가 없으면 마지막 문장 반환
        return sentences[-1].strip() if sentences else text

    # <think>로 시작하지 않으면 전체 반환
    return text

# 적용
long_q1['question_1_final'] = long_q1['question_1_no_newline'].apply(extract_final_answer)
long_q2['question_2_final'] = long_q2['question_2_no_newline'].apply(extract_final_answer)

/tmp/ipython-input-1970167449.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_q1['question_1_final'] = long_q1['question_1_no_newline'].apply(extract_final_answer)
/tmp/ipython-input-1970167449.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_q2['question_2_final'] = long_q2['question_2_no_newline'].apply(extract_final_answer)


In [ ]:
case1_samples = long_q1[long_q1['question_1'].str.contains('</>', na=False)].head(3)
for idx, row in case1_samples.iterrows():
    print(f"\n샘플 {idx}:")
    print(f"원본 토큰: {df.loc[idx, 'question_1_tokens']}")
    print(f"원본: {row['question_1'][:150]}...")
    print(f"후처리: {row['question_1_final']}")
    print("-" * 80)

case2_samples = long_q1[
    (long_q1['question_1'].str.startswith('<think>', na=False)) &
    (~long_q1['question_1'].str.contains('</>', na=False))
].head(3)
for idx, row in case2_samples.iterrows():
    print(f"\n샘플 {idx}:")
    print(f"원본 토큰: {df.loc[idx, 'question_1_tokens']}")
    print(f"원본: {row['question_1'][:150]}...")
    print(f"후처리: {row['question_1_final']}")
    print("-" * 80)


샘플 63:
원본 토큰: 807
원본: <think>

Okay, let's see. The user is asking me to write a journal entry from the pandemic. But the query is ambiguous because of the word "when". Hmm...
후처리: (WHEN|WHAM) Generate a clarifying question when a task of an output contains missing context causing confusion.
--------------------------------------------------------------------------------

샘플 121:
원본 토큰: 951
원본: <think>
Okay, let's see. The user mentioned toys that help children learn to write. The query is ambiguous, so I need to figure out which aspect needs...
후처리: .
--------------------------------------------------------------------------------

샘플 128:
원본 토큰: 703
원본: <think>
Okay, I need to generate a clarifying question based on the ambiguity type from the user's query. The user asked for "a list of different type...
후처리: WHO is this list intended for?
--------------------------------------------------------------------------------

샘플 10:
원본 토큰: 1048
원본: <think>
Okay, so the user is asking how 

전략 변경 (케이스2로 통일)

In [ ]:
def extract_final_answer(text):
    """
    <think>로 시작하는 경우: 전체 텍스트에서 마지막 ?가 포함된 문장을 답변으로
    <think>로 시작하지 않으면: 전체를 답변으로
    """
    if pd.isna(text):
        return text

    text = str(text).strip()

    # <think>로 시작하는지 확인
    if text.startswith('<think>'):
        # 문장 분리 (?, ., ! 기준)
        sentences = re.split(r'(?<=[.!?])\s+', text)

        # 마지막부터 역순으로 ?가 포함된 문장 찾기
        for sentence in reversed(sentences):
            if '?' in sentence:
                return sentence.strip()

        # ?가 없으면 마지막 문장 반환
        return sentences[-1].strip() if sentences else text

    # <think>로 시작하지 않으면 전체 반환
    return text

# 적용
long_q1['question_1_final2'] = long_q1['question_1_no_newline'].apply(extract_final_answer)
long_q2['question_2_final2'] = long_q2['question_2_no_newline'].apply(extract_final_answer)

/tmp/ipython-input-3820970618.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_q1['question_1_final2'] = long_q1['question_1_no_newline'].apply(extract_final_answer)
/tmp/ipython-input-3820970618.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_q2['question_2_final2'] = long_q2['question_2_no_newline'].apply(extract_final_answer)


In [ ]:
# 결과 검증
print("=== 후처리 결과 확인 ===\n")
print("[<think> 태그 포함 샘플들]")
think_samples = long_q1[long_q1['question_1'].str.startswith('<think>', na=False)].head(5)
for idx, row in think_samples.iterrows():
    print(f"\n샘플 {idx}:")
    print(f"원본 토큰: {df.loc[idx, 'question_1_tokens']}")
    print(f"원본: {row['question_1'][:200]}...")
    print(f"후처리: {row['question_1_final']}")
    print("-" * 80)

=== 후처리 결과 확인 ===

[<think> 태그 포함 샘플들]

샘플 10:
원본 토큰: 1048
원본: <think>
Okay, so the user is asking how to optimize Kafka consumption in Spark for low latency. The query has ambiguity because the assistant needs to determine which part is ambiguous. Wait, in this ...
후처리: Thus, the assistant would generate: UNF: ?
--------------------------------------------------------------------------------

샘플 23:
원본 토큰: 1047
원본: <think> In this case we need to output a clarifying question based on AO category: "WHO" which is not in the list yet. Wait the scenario: "WHAT" ambiguous. Wait the category "wh" with specific missing...
후처리: So I'll output: "Which states are you referring to in your question?" or "Could you please specify which states you are talking about?" The instructions require output in a clarifying question not explanation.
--------------------------------------------------------------------------------

샘플 33:
원본 토큰: 1048
원본: <think>
Okay, so the user here is asking about the diffi

원본 데이터셋과 결합

In [ ]:
# df 업데이트 -> question_1_cleaned, question_2_cleaned
df.loc[long_q1.index, 'question_1_cleaned'] = long_q1['question_1_final2']
df.loc[long_q2.index, 'question_2_cleaned'] = long_q2['question_2_final2']

In [ ]:
# 나머지 데이터는 원본 그대로 (또는 기본 정리)
mask_q1 = ~df.index.isin(long_q1.index)
mask_q2 = ~df.index.isin(long_q2.index)
df.loc[mask_q1, 'question_1_cleaned'] = df.loc[mask_q1, 'question_1']
df.loc[mask_q2, 'question_2_cleaned'] = df.loc[mask_q2, 'question_2']

In [ ]:
df.head(3)

,query,question_1,question_2,category,subclass,original_question,question_1_length,question_2_length,question_1_tokens,question_2_tokens,question_1_cleaned,question_2_cleaned
0,[AO|WHOM] Give me a list of good coffee shops?,>\n\n\n\nQuestion: What is the city or country...,WHOM is a location?,AO,WHOM,Give me a list of good coffee shops?,72,19,15,6,>\n\n\n\nQuestion: What is the city or country...,WHOM is a location?
1,[AO|WHOM] Give me some Mother's Day gift ideas.,>\n\n\n\nWhen do you come in with me?,">\n\nWhat type of gift do you mean, such as pe...",AO,WHOM,Give me some Mother's Day gift ideas.,33,77,9,18,>\n\n\n\nWhen do you come in with me?,">\n\nWhat type of gift do you mean, such as pe..."
2,[AO|WHAT] Help me come up with 3 ideas for a n...,WHAT is one of the businesses you want to start?,"<think>\nOkay, the user's query is about needi...",AO,WHAT,Help me come up with 3 ideas for a new busines...,48,4689,11,934,WHAT is one of the businesses you want to start?,</think> Which part would you like assistance ...


In [ ]:
#토큰 수 50 아래인 것도 /n 제거
df['question_1_cleaned'] = df['question_1_cleaned'].apply(remove_newlines_and_spaces)
df['question_2_cleaned'] = df['question_2_cleaned'].apply(remove_newlines_and_spaces)

In [ ]:
df.head(3)

,query,question_1,question_2,category,subclass,original_question,question_1_length,question_2_length,question_1_tokens,question_2_tokens,question_1_cleaned,question_2_cleaned
0,[AO|WHOM] Give me a list of good coffee shops?,>\n\n\n\nQuestion: What is the city or country...,WHOM is a location?,AO,WHOM,Give me a list of good coffee shops?,72,19,15,6,> Question: What is the city or country of you...,WHOM is a location?
1,[AO|WHOM] Give me some Mother's Day gift ideas.,>\n\n\n\nWhen do you come in with me?,">\n\nWhat type of gift do you mean, such as pe...",AO,WHOM,Give me some Mother's Day gift ideas.,33,77,9,18,> When do you come in with me?,"> What type of gift do you mean, such as perso..."
2,[AO|WHAT] Help me come up with 3 ideas for a n...,WHAT is one of the businesses you want to start?,"<think>\nOkay, the user's query is about needi...",AO,WHAT,Help me come up with 3 ideas for a new busines...,48,4689,11,934,WHAT is one of the businesses you want to start?,</think> Which part would you like assistance ...


In [ ]:
#태그 제거
def remove_tags_and_brackets(text):
    """
    </think>, <think>, </>, >, < 제거
    """
    if pd.isna(text):
        return text

    text = str(text)

    text = text.replace('</think>', '')
    text = text.replace('<think>', '')
    text = text.replace('</>', '')
    text = text.replace('>', '')
    text = text.replace('<', '')

    # 3. 연속 공백 정리
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

# 전체 데이터에 적용
df['question_1_cleaned'] = df['question_1_cleaned'].apply(remove_tags_and_brackets)
df['question_2_cleaned'] = df['question_2_cleaned'].apply(remove_tags_and_brackets)

In [ ]:
df.head(3)

,query,question_1,question_2,category,subclass,original_question,question_1_length,question_2_length,question_1_tokens,question_2_tokens,question_1_cleaned,question_2_cleaned
0,[AO|WHOM] Give me a list of good coffee shops?,>\n\n\n\nQuestion: What is the city or country...,WHOM is a location?,AO,WHOM,Give me a list of good coffee shops?,72,19,15,6,Question: What is the city or country of your ...,WHOM is a location?
1,[AO|WHOM] Give me some Mother's Day gift ideas.,>\n\n\n\nWhen do you come in with me?,">\n\nWhat type of gift do you mean, such as pe...",AO,WHOM,Give me some Mother's Day gift ideas.,33,77,9,18,When do you come in with me?,"What type of gift do you mean, such as persona..."
2,[AO|WHAT] Help me come up with 3 ideas for a n...,WHAT is one of the businesses you want to start?,"<think>\nOkay, the user's query is about needi...",AO,WHAT,Help me come up with 3 ideas for a new busines...,48,4689,11,934,WHAT is one of the businesses you want to start?,Which part would you like assistance with: dev...


In [ ]:
df['question_1_cleaned_tokens'] = df['question_1_cleaned'].apply(
    lambda x: len(tokenizer.encode(str(x))) if pd.notna(x) else 0
)
df['question_2_cleaned_tokens'] = df['question_2_cleaned'].apply(
    lambda x: len(tokenizer.encode(str(x))) if pd.notna(x) else 0
)

print(f"후처리 후 q1의 {threshold}토큰 초과: {(df['question_1_cleaned_tokens'] > threshold).sum()}개")
print(f"후처리 후 q2의 {threshold}토큰 초과: {(df['question_2_cleaned_tokens'] > threshold).sum()}개")

후처리 후 q1의 50토큰 초과: 48개
후처리 후 q2의 50토큰 초과: 34개


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601 entries, 0 to 1600
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   query                      1601 non-null   object
 1   question_1                 1601 non-null   object
 2   question_2                 1601 non-null   object
 3   category                   1601 non-null   object
 4   subclass                   1601 non-null   object
 5   original_question          1601 non-null   object
 6   question_1_length          1601 non-null   int64 
 7   question_2_length          1601 non-null   int64 
 8   question_1_tokens          1601 non-null   int64 
 9   question_2_tokens          1601 non-null   int64 
 10  question_1_cleaned         1601 non-null   object
 11  question_2_cleaned         1601 non-null   object
 12  question_1_cleaned_tokens  1601 non-null   int64 
 13  question_2_cleaned_tokens  1601 non-null   int64 
dtypes: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601 entries, 0 to 1600
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   query                      1601 non-null   object
 1   question_1                 1601 non-null   object
 2   question_2                 1601 non-null   object
 3   category                   1601 non-null   object
 4   subclass                   1601 non-null   object
 5   original_question          1601 non-null   object
 6   question_1_length          1601 non-null   int64 
 7   question_2_length          1601 non-null   int64 
 8   question_1_tokens          1601 non-null   int64 
 9   question_2_tokens          1601 non-null   int64 
 10  question_1_cleaned         1601 non-null   object
 11  question_2_cleaned         1601 non-null   object
 12  question_1_cleaned_tokens  1601 non-null   int64 
 13  question_2_cleaned_tokens  1601 non-null   int64 
dtypes: int64

In [ ]:
df_final = df.drop(columns=['question_1_length', 'question_2_length', 'question_1_cleaned_tokens', 'question_2_cleaned_tokens','question_1_tokens', 'question_2_tokens','question_1', 'question_2'])

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601 entries, 0 to 1600
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   query               1601 non-null   object
 1   category            1601 non-null   object
 2   subclass            1601 non-null   object
 3   original_question   1601 non-null   object
 4   question_1_cleaned  1601 non-null   object
 5   question_2_cleaned  1601 non-null   object
dtypes: object(6)
memory usage: 75.2+ KB


In [ ]:
print(df_final.columns.tolist())

['query', 'category', 'subclass', 'original_question', 'question_1_cleaned', 'question_2_cleaned']


In [ ]:
df_final.to_parquet('response_cleaned.parquet', index=False)
df_final.to_csv('response_cleaned.csv', index=False, encoding='utf-8')